# Working with Geospatial Data in Python

**Data Sources**

- [National Oceanic and Atmospheric Administration's Wrecks and Obstructions Database](https://nauticalcharts.noaa.gov/data/wrecks-and-obstructions.html): collection of known wrecks and obstructions in US coastal waters courtesy of the Coast Survey's Automated Wreck and Obstruction Information System (AWOIS)


In [45]:
# Package imports
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely

import matplotlib.pyplot as plt
# import contextily as ctx
import folium

%matplotlib inline

In [46]:
# Read AWOIS Wreck shapefile into GeoDataFrames
awois_wrecks = gpd.read_file('./data/AWOIS_Wrecks/AWOIS_Wrecks.shp', driver='shapefile')

# Keep only rows in geographic regions B and C (Southern MA to Northern NJ)
awois_wrecks = awois_wrecks[awois_wrecks['AREA_ID'].str.contains('B|C')]

awois_wrecks.head()

,RECRD,VESSLTERMS,AREA_ID,CHART,LATDEC,LONDEC,GP_QUALITY,GP_SOURCE,DEPTH,SOUNDING_T,YEARSUNK,HISTORY,REFERENCE,geometry
1093,15129,WRECK,C,12402,40.567114,-74.047717,High,Direct,4,Feet and tenths,None,"LNM09/12, USCG District 1-- Added ""4"" wreck an...",None,POINT (-74.04772 40.56711)
1094,8909,UNKNOWN,C,12214,38.845972,-74.835139,High,Direct,28,Feet and tenths,None,H-10241/94-- OPR-D368-WH; UNCHARTED WRECKAGE A...,None,POINT (-74.83514 38.84597)
1095,11992,UNKNOWN,C,12353,40.618333,-73.08025,High,Direct,50,Feet and tenths,None,\r\n HISTORY\r\n LNM28/90 (7/11/90)-- ADD SYM...,None,POINT (-73.08025 40.61833)
1096,12021,UNKNOWN,C,12214,38.928942,-74.855206,High,Direct,35,Feet and tenths,None,H11104/02--OPR-C303-KR; FOUND A SUNKEN WRECK ...,None,POINT (-74.85521 38.92894)
1097,12026,UNKNOWN,C,12214,38.903281,-74.814119,High,Direct,34,Feet and tenths,None,H11104/02--OPR-C303-KR; FOUND A SUNKEN WRECK ...,None,POINT (-74.81412 38.90328)


In [47]:
awois_wrecks.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1771 entries, 1093 to 5346
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   RECRD       1771 non-null   object  
 1   VESSLTERMS  1771 non-null   object  
 2   AREA_ID     1771 non-null   object  
 3   CHART       1765 non-null   object  
 4   LATDEC      1771 non-null   object  
 5   LONDEC      1771 non-null   object  
 6   GP_QUALITY  1750 non-null   object  
 7   GP_SOURCE   1477 non-null   object  
 8   DEPTH       1591 non-null   object  
 9   SOUNDING_T  845 non-null    object  
 10  YEARSUNK    16 non-null     object  
 11  HISTORY     1725 non-null   object  
 12  REFERENCE   28 non-null     object  
 13  geometry    1771 non-null   geometry
dtypes: geometry(1), object(13)
memory usage: 207.5+ KB


In [48]:
awois_wrecks['VESSLTERMS'].value_counts()

UNKNOWN         1012
WRECK            160
OBSTRUCTION        5
SHINNECOCK         4
SAN DIEGO          3
                ... 
ZURICHMOOR         1
CAPE MAY           1
SEACONNET          1
AMELIA             1
VALIANT LADY       1
Name: VESSLTERMS, Length: 567, dtype: int64

In [49]:
# Read AWOIS Obstructions shapefile into GeoDataFrames
awois_obs = gpd.read_file('./data/AWOIS_Obstructions/AWOIS_Obstructions.shp', driver='shapefile')

# Keep only rows in geographic regions B and C (Southern MA to Northern NJ)
awois_obs = awois_obs[awois_obs['AREA_ID'].str.contains('B|C')]

awois_obs.head()

,RECRD,VESSLTERMS,AREA_ID,CHART,LATDEC,LONDEC,GP_QUALITY,GP_SOURCE,DEPTH,SOUNDING_T,YEARSUNK,HISTORY,REFERENCE,geometry
1441,15204,OBSTRUCTION,C,12326,40.338361,-73.699722,None,Not Provided,24.7,Meters and tenths,None,H12627/OPR-B310-FH-13: New wreck identified at...,None,POINT (-73.69972 40.33836)
1442,8910,OBSTRUCTION,C,12214,38.821772,-74.829433,High,Direct,0,None,None,HISTORY\r\n H-10241/94-- OPR-D368-WH; UNCHART...,None,POINT (-74.82943 38.82177)
1443,8911,OBSTRUCTION,C,12214,38.840908,-74.837733,High,Direct,12.4,Meters and tenths,None,HISTORY\r\n H-10241/94-- OPR-D368-WH; UNCHART...,None,POINT (-74.83773 38.84091)
1444,8777,OBSTRUCTION,C,12214,38.803025,-74.947608,High,Direct,11.9,Meters and tenths,None,HISTORY\r\n H10444/92-93; FE-387/93-- OPR-D36...,None,POINT (-74.94761 38.80302)
1445,8778,OBSTRUCTION,C,12214,38.805506,-74.919508,High,Direct,11.5,Meters and tenths,None,HISTORY\r\n H10444/92-93; FE-387/93-- OPR-D36...,None,POINT (-74.91951 38.80551)


In [50]:
awois_obs.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 1218 entries, 1441 to 5274
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   RECRD       1218 non-null   object  
 1   VESSLTERMS  1218 non-null   object  
 2   AREA_ID     1218 non-null   object  
 3   CHART       1211 non-null   object  
 4   LATDEC      1218 non-null   object  
 5   LONDEC      1218 non-null   object  
 6   GP_QUALITY  1203 non-null   object  
 7   GP_SOURCE   1203 non-null   object  
 8   DEPTH       1074 non-null   object  
 9   SOUNDING_T  759 non-null    object  
 10  YEARSUNK    2 non-null      object  
 11  HISTORY     1208 non-null   object  
 12  REFERENCE   0 non-null      object  
 13  geometry    1218 non-null   geometry
dtypes: geometry(1), object(13)
memory usage: 142.7+ KB


In [51]:
# Read ENC Wrecks shapefile into GeoDataFrames
enc_wrecks = gpd.read_file('./data/ENC_Wrecks/ENC_Wrecks.shp', driver='shapefile')

enc_wrecks.head()

,OBJL,CATWRK,CONRAD,CONVIS,EXPSOU,HEIGHT,OBJNAM,QUASOU,SOUACC,TECSOU,...,VERACC,VERDAT,VERLEN,WATLEV,INFORM,SCAMIN,SORDAT,SORIND,DSNM,geometry
0,159.0,5,NaN,NaN,NaN,NaN,None,None,None,None,...,None,None,None,2.0,None,105000.0,20150717,None,US509890.000,POINT (-79.03783 9.56900)
1,159.0,5,NaN,NaN,NaN,NaN,None,None,None,None,...,None,None,None,2.0,None,105000.0,20150717,None,US509890.000,POINT (-78.87901 9.55749)
2,159.0,5,NaN,NaN,NaN,NaN,None,None,None,None,...,None,None,None,2.0,None,105000.0,20150717,None,US509890.000,POINT (-78.94357 9.55448)
3,159.0,5,NaN,NaN,NaN,NaN,None,None,None,None,...,None,None,None,2.0,None,37500.0,20140603,None,US510820.000,POINT (-72.54199 18.23128)
4,159.0,5,NaN,NaN,NaN,NaN,None,None,None,None,...,None,None,None,2.0,None,37500.0,20140603,None,US510820.000,POINT (-72.53418 18.22828)


In [52]:
enc_wrecks.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 11384 entries, 0 to 11383
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   OBJL      11384 non-null  float64 
 1   CATWRK    11342 non-null  object  
 2   CONRAD    4 non-null      float64 
 3   CONVIS    9 non-null      float64 
 4   EXPSOU    3998 non-null   float64 
 5   HEIGHT    4 non-null      float64 
 6   OBJNAM    0 non-null      object  
 7   QUASOU    8707 non-null   object  
 8   SOUACC    0 non-null      object  
 9   TECSOU    85 non-null     object  
 10  VALSOU    0 non-null      object  
 11  VERACC    0 non-null      object  
 12  VERDAT    0 non-null      object  
 13  VERLEN    0 non-null      object  
 14  WATLEV    11384 non-null  float64 
 15  INFORM    0 non-null      object  
 16  SCAMIN    1918 non-null   float64 
 17  SORDAT    11383 non-null  object  
 18  SORIND    0 non-null      object  
 19  DSNM      11384 non-null  object  
 20

In [53]:
# Remove points outside the general area of interest (southern MA to northern NJ)
#   Longitude extent for area of interest = [-74.95, -63.6]
#   Latitude extent for area of interest = [38.8, 41.8]

# Create a polygon using the lat/lon values
enc_extent = shapely.geometry.Polygon(((-63.6, 38.8),
                                      (-63.6, 41.8),
                                      (-74.95, 41.8),
                                      (-74.95, 38.8),
                                      (-63.6, 38.8)))

In [54]:
# Create mask for points of ENC wrecks that fall in polygon
in_extent = enc_wrecks['geometry'].within(enc_extent)

# Update GeoDataFrame keeping only those points
enc_wrecks = enc_wrecks[in_extent]
enc_wrecks.head()

,OBJL,CATWRK,CONRAD,CONVIS,EXPSOU,HEIGHT,OBJNAM,QUASOU,SOUACC,TECSOU,...,VERACC,VERDAT,VERLEN,WATLEV,INFORM,SCAMIN,SORDAT,SORIND,DSNM,geometry
823,159.0,2,NaN,NaN,1.0,NaN,None,6,None,None,...,None,None,None,3.0,None,NaN,20120807,None,US5CN10M.000,POINT (-73.17760 40.93377)
824,159.0,1,NaN,NaN,1.0,NaN,None,6,None,None,...,None,None,None,3.0,None,89999.0,20070720,None,US5CN10M.000,POINT (-73.17156 41.02972)
825,159.0,2,NaN,NaN,1.0,NaN,None,6,None,None,...,None,None,None,3.0,None,NaN,20120807,None,US5CN10M.000,POINT (-73.20218 40.94870)
826,159.0,1,NaN,NaN,1.0,NaN,None,6,None,None,...,None,None,None,3.0,None,89999.0,20070720,None,US5CN10M.000,POINT (-73.07275 41.04364)
827,159.0,2,NaN,NaN,1.0,NaN,None,6,None,None,...,None,None,None,3.0,None,NaN,20120610,None,US5CN10M.000,POINT (-73.26062 40.97505)


In [55]:
enc_wrecks.shape

(1930, 21)

In [56]:
# Read Biela shapefile into GeoDataFrames
biela = gpd.read_file('./data/Biela/Biela.shp', driver='shapefile')

biela.head()

,id,Name,descriptio,timestamp,begin,end,altitudeMo,tessellate,extrude,visibility,drawOrder,icon,gx_media_l,geometry
0,73,BIELA,"<img src=""https://doc-08-10-mymaps.googleuserc...",None,None,None,None,-1,0,-1,None,None,https://doc-08-10-mymaps.googleusercontent.com...,POINT Z (-70.91667 40.15000 0.00000)


In [57]:
# Convert geometry to 2D point to conform with other datasets
#   Extra Z dimension is common when data originate from KML files
biela.geometry = biela.geometry.map(lambda polygon: shapely.ops.transform(lambda x, y, z: (x, y), polygon))

biela.head()

,id,Name,descriptio,timestamp,begin,end,altitudeMo,tessellate,extrude,visibility,drawOrder,icon,gx_media_l,geometry
0,73,BIELA,"<img src=""https://doc-08-10-mymaps.googleuserc...",None,None,None,None,-1,0,-1,None,None,https://doc-08-10-mymaps.googleusercontent.com...,POINT (-70.91667 40.15000)


## Check and Convert Coordinate Reference Systems

When combining geospatial datasets, the coordinate reference systems for each set must match (otherwise, you'll introduce error). Geopandas makes checking and setting the CRS easy with the `.crs` attribute. It shows the EPSG code for a geoDataFrame's CRS - `4326` is WGS84, and `4269` is NAD83.

All analysis done in the rest of the notebook will be mapped with folium, which assumes datasets are in WGS84, so all GeoDataFrames are converted to this CRS. Most web tile providers typically use Spherical ("Web") Mercator  projection (EPSG code `3857`), but folium does the projection conversion under the hood automatically.

The [Spatial Reference website](www.spatialreference.org) is a good resource to look up EPSG codes.

In [58]:
awois_wrecks.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - NAD83
- bounds: (167.65, 14.92, -47.74, 86.46)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [59]:
awois_obs.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - NAD83
- bounds: (167.65, 14.92, -47.74, 86.46)
Datum: North American Datum 1983
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [60]:
enc_wrecks.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [61]:
biela.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [63]:
# Convert AWOIS datasets from NAD83 to WGS84

# Alternatives: gdf.crs = 'EPSG:4326' or gdf.to_crs('EPSG:4326', inplace=True)
awois_wrecks.to_crs(epsg=4326, inplace=True)
awois_obs.to_crs(epsg=4326, inplace=True)

# Confirm the conversion worked and all CRSs are same
print('AWOIS Wrecks CRS: {}'.format(awois_wrecks.crs))
print('AWOIS Obstructions CRS: {}'.format(awois_obs.crs))
print('ENC Wrecks CRS: {}'.format(enc_wrecks.crs))
print('Biela Wreck CRS: {}'.format(biela.crs))

AWOIS Wrecks CRS: epsg:4326
AWOIS Obstructions CRS: epsg:4326
ENC Wrecks CRS: epsg:4326
Biela Wreck CRS: epsg:4326


## Map Wreck and Obstruction Data

In [64]:
# Helper function to display a folium map in Jupyter notebook
def display_map(m, filename):
    """
    Helper code to ensure the folium map will display
        in different browsers when viewing the Jupyter
        notebook.
    Side effect: saves a local copy of the HTML version
        of the map.
    
    :param m: a folium map
    :param filename: str for map filename to save a copy locally
    :return: IFrame object displaying the saved map
    """
    from IPython.display import IFrame
    m.save(filename)
    return (IFrame(filename,
                   width='100%',
                   height='500px'))


In [65]:
# Latitude and longitude to center the map
map_lat = 40.5
map_lon = -71.5


b_lat = biela['geometry'].y
b_lon = biela['geometry'].x

In [66]:
# Create a map of known obstructions
m = folium.Map(location=[map_lat, map_lon],
               tiles="stamenterrain",
               zoom_start=8)

# Create and plot circles for AWOIS wrecks
for idx, row in awois_wrecks.iterrows():
    lat = row["geometry"].y
    lon = row["geometry"].x
    folium.Circle(location=[lat, lon],
                  radius=10,
                  color='blue').add_to(m)
#                   color=color_point(row["point_type"])).add_to(m)

# Create and plot a marker for the Biela
folium.Marker([b_lat, b_lon], popup='<b>BIELA</b>').add_to(m)

# Display map
display_map(m, 'index.html')

In [55]:
# Plot ENC Wrecks to verify extent worked

# Change CRS to Spherical (Web) Mercator
map_enc_wrecks = enc_wrecks['geometry']
map_enc_wrecks.to_crs(epsg=3857)

# Plot with basemap
# fig, ax = plt.subplots(figsize=(15, 15))
# map_enc_wrecks.plot(ax=ax, alpha=0.5, edgecolor='k')
# ctx.add_basemap(ax)

823      POINT (-73.17760 40.93377)
824      POINT (-73.17156 41.02972)
825      POINT (-73.20218 40.94870)
826      POINT (-73.07275 41.04364)
827      POINT (-73.26062 40.97505)
                    ...            
11182    POINT (-73.04294 39.54048)
11183    POINT (-70.43424 40.93197)
11184    POINT (-73.21523 39.65697)
11185    POINT (-72.81739 39.78337)
11186    POINT (-69.84477 40.14692)
Name: geometry, Length: 1930, dtype: geometry